# Differantial Private Deep Learning

When our neural network are learning from sensitive data, that thay are only learning what they're supposed to learn from the data without accidentally learning they are not supposed to learn from the data.

The general goal of differantial privacy is to ensure that different kind of statistical analysis do not compromise the privacy.

The defination of Privacy: You will not affected, adversly on otherwise, by allowing your data to be used in any study on analysis, no matter what other studies,datasets, orinformation sources, are avaible.


Exercise
* Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

*  I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

    * creates the initial database (db)
    * creates all parallel databases


In [1]:
#function to crete a databse
import torch
def remove_index(db,index):
    """Remove the defined index from database"""
    #removing the index
    return torch.cat((db[0:index],db[index + 1:]))

def paralle_db(db):
    """Create paralel data base for a given database with using remove_index function"""
    #create the empty list for paralel databases
    paralel_dbs = list()
    #loop over the database
    for i in range(len(db)):
        #create the database for each removed index
        dbs = remove_index(db,i)
        #appande the removed databases to list of paralel databases
        paralel_dbs.append(dbs)
    return paralel_dbs

def create_and_paralel(num_enrty):
    """Create simple database for a single column and then call the parael_db to
    create paralal databases for each removed index value"""
    #create the simple database for specific number 
    db = torch.rand(num_enrty) > 0.5
    #call paralle_bd function to create paralal databases for spefic number of element
    pbds = paralle_db(db)
    return db, pbds
    

In [2]:
bd, pdbs = create_and_paralel(5000)

In [3]:
#let's check out first 6 tenser list from pdbs
print(pdbs[0:5])

[tensor([1, 0, 1,  ..., 0, 0, 0], dtype=torch.uint8), tensor([1, 0, 1,  ..., 0, 0, 0], dtype=torch.uint8), tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8), tensor([1, 1, 0,  ..., 0, 0, 0], dtype=torch.uint8), tensor([1, 1, 0,  ..., 0, 0, 0], dtype=torch.uint8)]


# Evaluating the The Privacy of the Function

*  How to query this database
* then measure the privacy of that query

We are going to compare the output of the query on the entire database, with the output of the query on each of the paralel databases. That way, we are going to how the query change, when we remove an individual from tha database.

* We are going to query the full database
* Weare going to look at quering every possible paralel database
* What's the maxmimum around that they different?

When we remove the people from this database, it changes the output of query. the output of this queryis actually conditionied directy on information from a lot of people in this databse.

Sensitivity: The maximum amount that the query changes when removing an individual from the database. it is called L1 sensitivity for simply.




In [4]:
def query(db):
    return db.sum()

In [5]:
db, pdbs = create_and_paralel(5000)

In [6]:
full_db_result = query(db)

In [7]:
sensitivy = 0
for dbs in pdbs:
    pdb_result = query(dbs)
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if (db_distance > sensitivy):
        sensitivy = db_distance
    
sensitivy

tensor(1)

In [8]:
def query(db):
    return db.float().mean()

def sensitivity(query, n_entries =1000):
    db, pdbs = create_and_paralel(n_entries)
   
    full_db_result = query(db)
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > sensitivity):
            sensitivity = db_distance
    
    return sensitivity

In [9]:
sensitivity(query)

tensor(0.0005)

# Local & Global Differantial Privacy
**Local differantial privacy:** Add noise to the function dat points
**Global Differantial pricacy:** Add noise on the output query on the database

## Local Differantial Privacy

Where given a collection of individual, each individial adds noise to the their data before sending it to the statistical database itself. So, everything that gets through is alrready noised. So, the protection is happening at the local level. 

Differantial privacy always requires a form of randomness or noise added to the quey to protec from like differnatial attack.

**Ransomized Responce:** Techniquie that is used social sciences when tring to a learn about the high level trends for a taban behavior.

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

**Flip a coin 2 times.**
* If the first coin flip is heads, answer honestly
* If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained.

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results.



In [106]:
db, pdbs = create_and_paralel(100)
#db
first_coin_flip = (torch.rand(len(db))> 0.5).float()
#first_coin_flip
second_coin_flip = (torch.rand(len(db))> 0.5).float()
augmented_databse = (db.float() * first_coin_flip) + ((1-first_coin_flip) * second_coin_flip)
augmented_databse
torch.mean(augmented_databse).float() *2 -0.5

tensor(0.6200)

**in the video**

$augmented\_databse\_mean = (true\_mean *noise) + (noise\_dist\_mean*(1-noise))$

**it is also equel to**

$augmented\_databse\_mean = (true\_mean *(1-noise)) + (noise\_dist\_mean* noise)$

**changing the $noise$ and $(1-noise)$ are the same...**

So, if we try to get $true\_mean$ here

**Note: $augmented\_databse\_mean is alse called sk_result which we are calculating before. So, I am going to continue with sk_result**

$ sk\_result =(true\_mean *(1-noise)) + (noise\_dist\_mean* noise)$

we want to $true\_mean$ alone. So, I want to take (1-noise) paranteses for both term

$sk\_result = (1-noise)(true\_mean + noise\_dist\_mean* (\frac{noise}{1-noise}) )$

$ \frac{sk\_result}{(1-noise)} =  true\_mean + noise\_dist\_mean * (\frac{noise}{1-noise})$

 $true\_mean = \frac{sk\_result}{(1-noise)} - noise\_dist\_mean * (\frac{noise}{1-noise}) $
 
**S0, we can take the $\frac{noise}{(1-noise)}$ parantehesis**

$true\_mean = (\frac{noise}{(1-noise)}) * (\frac{sk\_result}{noise} - noise\_dist\_mean)  $

we know that $noise\_dist\_mean = 0.5$

$true\_mean = (\frac{noise}{(1-noise)}) * (\frac{sk\_result}{noise} - 0.5)  $

$true\_mean = ((sk\_learn / noise) - 0.5)* noise/(1-noise) $





